In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [29]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords

In [30]:
col_name = ["no", "name", "result", "review"]
df1 = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv", names=col_name)
df2 = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv", names=col_name)
df = pd.concat([df1, df2])

In [31]:
df.isnull().sum()

no          0
name        0
result      0
review    686
dtype: int64

In [32]:
df=df.dropna()

In [33]:
df.isnull().sum()

no        0
name      0
result    0
review    0
dtype: int64

In [34]:
df = df.drop("no", axis=1)
df = df[~df['result'].isin(['Neutral', 'Irrelevant'])]


print(df.head())

                   name    result  \
2             Microsoft  Negative   
3                 CS-GO  Negative   
5                  FIFA  Negative   
6             MaddenNFL  Positive   
7  TomClancysRainbowSix  Positive   

                                              review  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet hacking,...  
5  Hi @EAHelp I’ve had Madeleine McCann in my cel...  
6  Thank you @EAMaddenNFL!! \n\nNew TE Austin Hoo...  
7  Rocket League, Sea of Thieves or Rainbow Six: ...  


In [35]:
mapping = {"Positive": 1, "Negative": 0}
df["result"] = df["result"].map(mapping)


In [36]:
def clean_html(text):
    clean = re.compile("<.*?>")
    return re.sub(clean, "", text)


df["review"] = df["review"].apply(clean_html)

In [37]:
def convert_lower(text):
    return text.lower()


df["review"] = df["review"].apply(convert_lower)



In [38]:
def remove_special(text):
    clean_text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    return clean_text


df["review"] = df["review"].apply(remove_special)

In [39]:
nltk.download('stopwords')

def remove_stopwords(text):
    stopwords_list = stopwords.words("english")
    tokens = text.split()
    clean_tokens = [token for token in tokens if token not in stopwords_list]
    return " ".join(clean_tokens)


df["review"] = df["review"].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
from nltk.stem import PorterStemmer


porter = PorterStemmer()


def stem_words(text):
    tokens = text.split()
    stemmed_tokens = [porter.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)


df["review"] = df["review"].apply(stem_words)

In [41]:
df["new_review"] = df["name"] + "-" + df["review"]


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["new_review"], df["result"], test_size=0.2, random_state=42)


In [43]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [44]:
from sklearn.ensemble import RandomForestClassifier


In [45]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

In [46]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9676308539944903


In [47]:
from sklearn.metrics import classification_report
cl=classification_report(y_test,y_pred)

In [49]:
print(cl)

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4583
           1       0.97      0.96      0.97      4129

    accuracy                           0.97      8712
   macro avg       0.97      0.97      0.97      8712
weighted avg       0.97      0.97      0.97      8712

